In [5]:
%reset -sf
import pandas as pd
import numpy as np
from datetime import datetime, date
from IPython.display import Markdown as md
# from iteration_utilities import flatten # can be probably removed from conda environment

In [6]:
url = "https://opendata.dwd.de/"
path = 'climate_environment/CDC/observations_germany/climate/daily/kl/'
recent_path = path + 'recent/'
historical_path = path + 'historical/'
filename = 'KL_Tageswerte_Beschreibung_Stationen.txt' 
ws = pd.read_csv(url + recent_path + filename, sep="\t", header=0, skiprows = 0, encoding = "ISO-8859-1").dropna()
ws.drop(0, inplace=True)

# save original column names
colnames = ws.columns[0].split(' ')

# rename column for instance to 'dummy'
ws.columns = ['dummy']

# split string up to 6th column
ws = ws['dummy'].str.split('\s+', n=6, expand=True)

# convert 1:6 to numeric
for col in  ws.iloc[:,1:6]:
    ws[col] = pd.to_numeric(ws[col], errors='coerce') 
    
# concat columns back to a type consistent dataframe
wst = pd.concat([ws.iloc[:,0:6], ws[6].str.slice(0,41), ws[6].str.slice(41,)], axis=1)  
wst.columns = colnames
wst.head()

,Stations_id,von_datum,bis_datum,Stationshoehe,geoBreite,geoLaenge,Stationsname,Bundesland
1,00001,19370101,19860630,478,47.8413,8.8493,Aach,Baden-...
2,00003,18910101,20110331,202,50.7827,6.0941,Aachen,Nordrh...
3,00011,19800901,20231204,680,47.9736,8.5205,Donaueschingen (Landeplatz),Baden-...
4,00044,19690101,20231204,44,52.9336,8.2370,Großenkneten,Nieder...
5,00052,19690101,20011231,46,53.6623,10.1990,Ahrensburg-Wulfsdorf,Schles...


In [7]:
wst.contains

,Stations_id,von_datum,bis_datum,Stationshoehe,geoBreite,geoLaenge,Stationsname,Bundesland
1,00001,19370101,19860630,478,47.8413,8.8493,Aach,Baden-...
2,00003,18910101,20110331,202,50.7827,6.0941,Aachen,Nordrh...
3,00011,19800901,20231204,680,47.9736,8.5205,Donaueschingen (Landeplatz),Baden-...
4,00044,19690101,20231204,44,52.9336,8.2370,Großenkneten,Nieder...
5,00052,19690101,20011231,46,53.6623,10.1990,Ahrensburg-Wulfsdorf,Schles...
...,...,...,...,...,...,...,...,...
1351,19631,18810601,19691231,268,50.8083,10.2294,Salzungen,Thürin...
1352,19647,19510101,20051031,178,49.4547,8.9794,Eberbach/Neckar,Baden-...
1353,19774,19710819,19940228,174,51.4814,10.8057,Nordhausen (Umspannwerk),Thürin...
1354,19781,19410101,19531231,367,48.7429,11.4233,Ingolstadt,Bayern...


In [101]:
# Convert von_datum and bis_datum to a date type
wst["von"] = pd.to_datetime(wst["von_datum"], format="%Y%m%d", errors="coerce")
wst["bis"] = pd.to_datetime(wst["bis_datum"], format="%Y%m%d", errors="coerce")
# print(wst.loc[:,['Stations_id','von','bis']])

# Function to extract stations Climatic Standard Normal periods, s. WMO-No. 1203
def dwd_extratCSN(df, yyyy, wmo1203=30):
    first = str(yyyy-wmo1203+1) + '-1-1'
    last = str(yyyy)+ '-12-31'
    fv = pd.to_datetime(first, format='%Y-%m-%d', errors="coerce").date()
    lv = pd.to_datetime(last, format='%Y-%m-%d', errors="coerce").date()
    return df[(df['von'] <= first) & (df['bis'] >= last)], (lv - fv).days

# Select stations having data for the Standard Reference Period and some Climatic Standard Normal periods
wmo_yyyy = 1990
CSN, CSN_days = dwd_extratCSN(wst.drop(['von_datum','bis_datum'], axis=1), wmo_yyyy)  # Standard Ref. Period
# print(CSN.loc[:,['Stations_id','von','bis']])
# print(CSN.describe())
# print(len(CSN.index))
# print(CSN_days)

### DWD Compare Each Month vs. Climatic Norm

In [102]:
md(f"{len(wst.index)} available weather stations {len(CSN.index)} of which contribute to the Climatic Standard Normal ending in {wmo_yyyy}. This period contains {CSN_days} days.")


1355 available weather stations 428 of which contribute to the Climatic Standard Normal ending in 1990. This period contains 10956 days.

In [103]:
# CSN.drop(['Stationshoehe','geoBreite','geoLaenge'], axis=1)
# print(CSN.loc[:,['Stations_id','Stationsname']].head())
# print(CSN.loc[:,['Stations_id','Stationsname']].tail(11).to_markdown())
CSN

,Stations_id,Stationshoehe,geoBreite,geoLaenge,Stationsname,Bundesland,von,bis
2,00003,202,50.7827,6.0941,Aachen,Nordrh...,1891-01-01,2011-03-31
10,00073,374,48.6183,13.0620,Aldersbach-Kramersepp,Bayern...,1959-03-01,2023-12-03
11,00078,64,52.4853,7.9125,Alfhausen,Nieder...,1961-01-01,2023-12-03
25,00142,511,48.4060,11.3117,Altomünster-Maisbrunn,Bayern...,1955-01-01,2023-12-03
26,00150,215,49.7273,8.1164,Alzey,Rheinl...,1951-01-01,2023-12-03
...,...,...,...,...,...,...,...,...
1327,15963,40,51.3258,6.5089,St. Tönis,Nordrh...,1954-01-01,2004-11-30
1334,19087,645,48.8049,13.5528,Freyung vorm Wald,Bayern...,1957-05-01,1995-11-30
1348,19582,754,48.1419,8.4255,Königsfeld,Baden-...,1951-01-01,2001-08-31
1350,19617,310,49.7391,10.6039,Burghaslach,Bayern...,1941-01-01,2006-12-31


Display number of weather stations by Bundesland:

In [104]:
print(CSN.groupby(['Bundesland'])['Bundesland'].count().to_markdown())

| Bundesland             |   Bundesland |
|:-----------------------|-------------:|
| Baden-Württemberg      |           78 |
| Bayern                 |           79 |
| Berlin                 |            3 |
| Brandenburg            |           13 |
| Bremen                 |            2 |
| Hamburg                |            4 |
| Hessen                 |           38 |
| Mecklenburg-Vorpommern |           16 |
| Niedersachsen          |           40 |
| Nordrhein-Westfalen    |           36 |
| Rheinland-Pfalz        |           27 |
| Saarland               |            5 |
| Sachsen                |           23 |
| Sachsen-Anhalt         |           22 |
| Schleswig-Holstein     |           14 |
| Thüringen              |           28 |


Download zip file from URL
[howto](https://pythonguides.com/download-zip-file-from-url-using-python/)

Loop zip files in zip_url and extract observations

In [105]:
from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile
from re import compile

def collectRecords(zfile): 
    with BytesIO(zfile.read()) as b, ZipFile(b) as datafile: 
        r = compile("^produkt_klima_tag_.*\.txt$")
        dfound = list(filter(r.match, datafile.namelist()))
        number = len(dfound)
        assert number == 1, f"WARN: exactly one element expected, got {number} instead"
        # print(dfound[0])
        rf = datafile.open(dfound[0])
        lines = rf.readlines()
        rf.close()
        header = True
        for bline in lines:
            line = bline.decode('unicode-escape').rstrip('\r\n').split(';')
            del line[-1] # remove last column containing only string 'eor'
            #print(line)
            if header: # initialize list of lists
                header = not(header)
                record = [line]
            else:
                record.append(line)
    return record

# Function to extract data for Climatic Standard Period
# See WMO Guidelines on the Calculation of Climate Normals: WMO-No. 1203
def dwd_extratCSNData(df, yyyy, wmo1203=30):
    first = str(yyyy-wmo1203+1) + '-1-1'
    last = str(yyyy)+ '-12-31'
    fv = pd.to_datetime(first, format='%Y-%m-%d', errors="coerce").date()
    lv = pd.to_datetime(last, format='%Y-%m-%d', errors="coerce").date()
    return df[(df['DATUM'] >= fv) & (df['DATUM'] <= lv)], (lv - fv).days

%time 
csn_lst = CSN['Stations_id'].to_list()

CPU times: user 9 µs, sys: 0 ns, total: 9 µs
Wall time: 16 µs


In [106]:
%%time

# zip_url = url + recent_path 
# tw_regex = compile(r'tageswerte_KL_[0-9]{5}_akt.zip')

zip_url = url + historical_path 
tw_regex = compile(r'tageswerte_KL_[0-9]{5}_[0-9]{8}_[0-9]{8}_hist.zip')

once = True
current_year = date.today().year # to end the csn_period loop
csn_dict = {}
with urlopen(zip_url) as f:
    for bline in f.readlines():
        zfound = tw_regex.search(bline.decode('utf-8'))
        # print(zfound)
        if zfound:
            zfilename = zfound.string[zfound.start():zfound.end()]
            sid = zfilename[14:19] # extracted weather station id
            
            # if sid in csn_lst[:3]:
            if sid in csn_lst:
                # print(sid)
                csn_dict[sid] = {}
                with urlopen(zip_url + zfilename) as z:
                    record = collectRecords(z) # <-- function call   
                    df = pd.DataFrame(record[1:], columns=record[0])

                    # Remove leading and trailing blanks from column names
                    trimmed = [s.strip() for s in list(df)]
                    df.columns = trimmed
                    df = df.apply(pd.to_numeric)
 
                    # Extract overall summary statistics
                    # print(df.describe())
                    csn_dict[sid]['all'] = {'summary': df.describe()}
                    
                    df["DATUM"] = pd.to_datetime(df["MESS_DATUM"], format="%Y%m%d", errors="coerce").dt.date
                    df["DAY"] = pd.to_datetime(df["MESS_DATUM"], format="%Y%m%d", errors="coerce").dt.day
                    df["MONTH"] = pd.to_datetime(df["MESS_DATUM"], format="%Y%m%d", errors="coerce").dt.month
                    df.drop(['MESS_DATUM'], axis='columns',inplace=True)
                    df.replace(-999, np.NaN, inplace=True)  
                    df['STATIONS_ID'] = df['STATIONS_ID'].astype('str').apply(lambda x: '{0:0>5}'.format(x))

                    for csn_period in range(1990, current_year, 10):
                        CSN_df, CSN_days = dwd_extratCSNData(df, csn_period)
                        season = df.groupby(['MONTH', 'DAY'])[['VPM', 'PM', 'RSK', 'TMK', 'UPM']].describe()
                        # season = df.groupby(['MONTH', 'DAY'])['TMK'].describe()
                        csn_dict[sid][csn_period] = {'days': CSN_days, 'summary': CSN_df.describe(), 'season': season}

md(f"{len(csn_dict)} DWD weather stations have been processed at URL {zip_url}.")
# print(zip_url)

CPU times: user 1h 20min 20s, sys: 3.65 s, total: 1h 20min 23s
Wall time: 1h 26min 36s


420 DWD weather stations have been processed at URL https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/daily/kl/historical/.

In [3]:
import pickle
dump_fn = 'dwd_csn.pickle'

# with open(dump_fn, 'wb') as f:
#     # Pickle the dictionary using the highest protocol available.
#     pickle.dump(csn_dict, f, pickle.HIGHEST_PROTOCOL)

with open(dump_fn, 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    csn_dict = pickle.load(f)

In [4]:
csn_dict.keys() # STATIONS_ID

dict_keys(['00003', '00073', '00078', '00142', '00150', '00151', '00164', '00183', '00186', '00198', '00217', '00232', '00243', '00257', '00259', '00268', '00282', '00298', '00314', '00320', '00330', '00361', '00377', '00379', '00400', '00403', '00427', '00433', '00445', '00450', '00460', '00480', '00524', '00535', '00554', '00555', '00591', '00596', '00599', '00614', '00619', '00650', '00656', '00662', '00691', '00701', '00717', '00722', '00736', '00755', '00817', '00851', '00853', '00863', '00867', '00880', '00881', '00891', '00896', '00953', '00963', '00979', '00982', '00991', '00998', '01001', '01018', '01032', '01048', '01050', '01051', '01053', '01072', '01076', '01078', '01079', '01087', '01091', '01161', '01182', '01197', '01201', '01207', '01219', '01229', '01238', '01254', '01270', '01279', '01297', '01300', '01303', '01327', '01329', '01346', '01357', '01358', '01379', '01410', '01411', '01420', '01441', '01443', '01467', '01468', '01490', '01492', '01503', '01526', '01544',